In [17]:
#CONNECTION 
import pandas as pd
import mysql.connector
import json
def Connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Rishi@12584",
        database="firstproject"
 )
db = Connection()
cursor = db.cursor()

In [18]:
#FUNCTION TO CREATE TABLES
def create_tables(cursor,db):
    try:
        for query in create_table_queries:
            cursor.execute(query)
            db.commit()
    except Exception as e:
        print(f"Error creating tables: {e}")

In [19]:
# LOADING DATA
customer_data = pd.read_json("customers.json")
transaction_data = pd.read_json("transaction_logs.json")
df = pd.DataFrame(customer_data)

In [20]:
customer_data.head()

transaction_data.head()

,order_id,customer_id,product_id,product_name,product_category,payment_type,qty,price,datetime,ecommerce_website_name,payment_txn_id,payment_txn_success,failure_reason
0,1,1731,8609,Central home & kitche,Home & Kitchen,Wallet,25,64216,2021-07-26 20:16:14,www.bestbuy.com,a395e8c7,N,Card Expired
1,2,5539,2409,Upon sport,Sports,Card,34,69877,2021-08-25 03:41:32,www.flipkart.com,b40791f8,N,Invalid CVV
2,3,3507,1145,Across stationer,Stationery,Card,28,35491,2024-06-21 21:39:21,www.target.com,7fb6cca9,N,Blocked Card
3,4,4826,8375,Start home & kitche,Home & Kitchen,Card,33,39540,2022-09-17 13:52:11,www.amazon.in,cae677f3,N,Transaction Timeout
4,5,6538,2149,Draw beaut,Beauty,Card,27,80635,2022-03-09 07:58:08,www.etsy.com,fbce1649,N,Invalid CVV


In [21]:
#CREATING TABLES
#CUSTOMERS TABLE
customer_table_create = """
CREATE TABLE IF NOT EXISTS Customers(
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    country VARCHAR(100),
    city VARCHAR(100)
);
"""
cursor.execute(customer_table_create)
db.commit()

#PRODUCTS TABLE
product_table_create = """
CREATE TABLE IF NOT EXISTS Products(
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    product_category VARCHAR(100),
    price DECIMAL(10, 2),
    ecommerce_website_name VARCHAR(255)
);
"""
cursor.execute(product_table_create)
db.commit()

#CUSTOMER DEMOGRAPHIC TABLE
customer_demographics_table_create = """
CREATE TABLE IF NOT EXISTS CustomerDemographic(
    customer_id INT PRIMARY KEY,
    age_group VARCHAR(50),
    gender CHAR(1),
    income_bracket VARCHAR(50),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
"""
cursor.execute(customer_demographics_table_create)
db.commit()

#CUSTOMER ADDRESS
customer_address_table_create = """
CREATE TABLE IF NOT EXISTS CustomerAddress (
    customer_id INT PRIMARY KEY,
    country VARCHAR(100),
    city VARCHAR(100),
    street_address VARCHAR(255),
    postal_code VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
"""
cursor.execute(customer_address_table_create)
db.commit()

#PRODUCT CATEGORY
product_category_table_create = """
CREATE TABLE IF NOT EXISTS ProductCategory (
    category_id SERIAL PRIMARY KEY,
    category_name VARCHAR(100) UNIQUE
);
"""
cursor.execute(product_category_table_create)
db.commit()

In [22]:
#INSERTING DATA INTO CUSTOMER TABLE
customer_insert_query = """
INSERT INTO Customers(customer_id, customer_name, country, city)
VALUES (%s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
customer_name = VALUES(customer_name),
country = VALUES(country),
city = VALUES(city);
"""
try:
    for i, row in customer_data.iterrows():
        cursor.execute(customer_insert_query, list(row))
    db.commit()     
    print("Data inserted into Customer table successfully.")
except Exception as e:
    print(f"Error inserting into Customer table: {e}")

Data inserted into Customer table successfully.


In [23]:
#INSERTING DATA INTO CUSTOMERDEMOGRAPHIC TABLE
customer_demographics_insert_query = """
INSERT INTO CustomerDemographic(customer_id, age_group, gender, income_bracket)
VALUES (%s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
age_group = VALUES(age_group),
gender = VALUES(gender),
income_bracket = VALUES(income_bracket);
"""
try:
    for i, row in customer_data.iterrows():
        cursor.execute(customer_demographics_insert_query, [row['customer_id'], "25-34", "F", "Medium"])
    db.commit()
    print("Data inserted into CustomerDemographics table successfully.")
except Exception as e:
    print(f"Error inserting into CustomerDemographics table: {e}")

Data inserted into CustomerDemographics table successfully.


In [24]:
#INSERTING DATA INTO CUSTOMER ADDRESS TABLE
customer_address_insert_query = """
INSERT INTO CustomerAddress (customer_id, country, city, street_address, postal_code)
VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
country = VALUES(country),
city = VALUES(city),
street_address = VALUES(street_address),
postal_code = VALUES(postal_code);
"""

# Insert data into CustomerAddress table (assuming you add street_address and postal_code)
try:
    for i, row in customer_data.iterrows():
        cursor.execute(customer_address_insert_query, [row['customer_id'], row['country'], row['city'], "Unknown Street", "00000"])
    db.commit()
    print("Data inserted into CustomerAddress table successfully.")
except Exception as e:
    print(f"Error inserting into CustomerAddress table: {e}")

Data inserted into CustomerAddress table successfully.


In [25]:
#INSERTING DATA INTO PRODUCT CATEGORY TABLE
product_category_insert_query = """
INSERT INTO ProductCategory (category_name)
VALUES (%s)
ON DUPLICATE KEY UPDATE
category_name = VALUES(category_name);
"""

# Insert data into ProductCategory table
try:
    unique_categories = transaction_data['product_category'].unique()
    for category in unique_categories:
        cursor.execute(product_category_insert_query, [category])
    db.commit()
    print("Data inserted into ProductCategory table successfully.")
except Exception as e:
    print(f"Error inserting into ProductCategory table: {e}")

Data inserted into ProductCategory table successfully.


In [26]:
##INSERTING DATA INTO PRODUCT TABLE
product_insert_query = """
INSERT INTO Products (product_id, product_name, product_category)
VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE 
    product_name = VALUES(product_name),
    product_category = VALUES(product_category);
"""

# Insert data into Product table
try:
    for i, row in transaction_data.iterrows():
        cursor.execute(product_insert_query, [row['product_id'], row['product_name'], row['product_category']])
    db.commit()
    print("Data inserted into Product table successfully.")
except Exception as e:
    print(f"Error inserting into Product table: {e}")


Data inserted into Product table successfully.


In [27]:
#RETRIVING DATA FROM THE TABLES
import mysql.connector
def fetch_sample_data(table_name, connection):
    try:
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(query)
        result = cursor.fetchall()
        print(f"\nData limited to 5 rows from {table_name} table:")
        for row in result:
            print(row)
    except mysql.connector.Error as err:
        print(f"Error: {err}")

def main():
    tables = ["Customers", "Products", "CustomerDemographic", "CustomerAddress","ProductCategory "]
    for table in tables:
        fetch_sample_data(table, db)

    # Close connection
    db.close()

if __name__ == "__main__":
    main()



Data limited to 5 rows from Customers table:
(102, 'Kenneth Singh', 'Sweden', 'Davidhaven')
(106, 'Mr. James Mendez', 'British Indian Ocean Territory (Chagos Archipelago)', 'Paulmouth')
(109, 'Joseph Martin', 'Somalia', 'Port Vickiemouth')
(135, 'Joann Romero', 'Timor-Leste', 'Port Raymondmouth')
(144, 'Robert Reeves', 'Wallis and Futuna', 'Lake Jason')
(145, 'Jasmine Dean', 'Tuvalu', 'Scotthaven')
(147, 'Charles Becker', 'Isle of Man', 'East Cindyfurt')
(158, 'Anthony Bowman', 'Latvia', 'Carriebury')
(159, 'Desiree Edwards', 'Central African Republic', 'North Williammouth')
(161, 'Kyle Lee', 'Greece', 'Jamesberg')

Data limited to 5 rows from Products table:
(201, 'Song toy', 'Toys', None, None)
(202, 'Enjoy stationer', 'Stationery', None, None)
(203, 'Management electronic', 'Electronics', None, None)
(206, 'Law electronic', 'Electronics', None, None)
(208, 'Issue electronic', 'Electronics', None, None)
(209, 'Pressure stationer', 'Stationery', None, None)
(210, 'Send electronic', '